In [6]:
pip install pymupdf

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
    --------------------------------------- 0.0/3.2 MB 1.9 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/3.2 MB 2.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.4/3.2 MB 3.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.8/3.2 MB 4.3 MB/s eta 0:00:01
   ------------- -------------------------- 1.1/3.2 MB 5.3 MB/s eta 0:00:01
   ----------------- ---------------------- 1.4/3.2 MB 5.2 MB/s eta 0:00:01
   ---------------------------- ----------- 2.3/3.2 MB 7.6 MB/s eta 0:00:01
   --------------------------------- ------ 2.7/3.2 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   - -------------------------------------- 0.4/12.5 MB 13.4 MB/s eta 0:00:01
   -- -----------------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

# Paths to the PDF files
alphabet_pdf_path = "C:/Users/pooja/Downloads/goog-10-k-2023 (1).pdf"
tesla_pdf_path = "C:/Users/pooja/Downloads/tsla-20231231-gen.pdf"
uber_pdf_path = "C:/Users/pooja/Downloads/uber-10-k-2023.pdf"

# Extracting text from each PDF
alphabet_text = extract_text_from_pdf(alphabet_pdf_path)
tesla_text = extract_text_from_pdf(tesla_pdf_path)
uber_text = extract_text_from_pdf(uber_pdf_path)

# Saving the extracted text to files for easier inspection and use
with open("alphabet_10k_text.txt", "w", encoding="utf-8") as f:
    f.write(alphabet_text)

with open("tesla_10k_text.txt", "w", encoding="utf-8") as f:
    f.write(tesla_text)

with open("uber_10k_text.txt", "w", encoding="utf-8") as f:
    f.write(uber_text)

print("Text extraction complete.")

Text extraction complete.


In [8]:
import re

def split_into_sections(text, sections):
    section_dict = {}
    for i in range(len(sections) - 1):
        pattern = f'{sections[i]}(.*?)(?={sections[i+1]})'
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            section_dict[sections[i]] = match.group(1).strip()
    # Handling the last section separately
    last_section = sections[-1]
    last_pattern = f'{last_section}(.*)'
    last_match = re.search(last_pattern, text, re.DOTALL | re.IGNORECASE)
    if last_match:
        section_dict[last_section] = last_match.group(1).strip()
    return section_dict

sections = [
    "Item 1. Business", 
    "Item 1A. Risk Factors", 
    "Item 2. Properties", 
    "Item 3. Legal Proceedings", 
    "Item 4. Mine Safety Disclosures",
    "Item 5. Market for Registrant's Common Equity",
    "Item 6. Selected Financial Data",
    "Item 7. Management's Discussion and Analysis",
    "Item 8. Financial Statements and Supplementary Data"
]

# Split each document into sections
alphabet_sections = split_into_sections(alphabet_text, sections)
tesla_sections = split_into_sections(tesla_text, sections)
uber_sections = split_into_sections(uber_text, sections)

print("Text organization into sections complete.")

Text organization into sections complete.


In [10]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def get_risk_factors(section_text):
    if section_text:
        doc = nlp(section_text)
        risk_factors = [sent.text for sent in doc.sents if "risk" in sent.text.lower()]
        return risk_factors
    return []

def compare_business_overview(overview_text1, overview_text2):
    if overview_text1 and overview_text2:
        words1 = Counter(overview_text1.split())
        words2 = Counter(overview_text2.split())
        common_words = words1 & words2  # Intersection of word counts
        return common_words
    return {}

# Example usage:
# Assume `alphabet_sections`, `tesla_sections`, and `uber_sections` are dictionaries containing
# the text of the different sections of the respective 10-K reports.

# Extract risk factors for Alphabet and Tesla
alphabet_risk_factors = get_risk_factors(alphabet_sections.get("Item 1A. Risk Factors", ""))
tesla_risk_factors = get_risk_factors(tesla_sections.get("Item 1A. Risk Factors", ""))

# Compare business overviews of Tesla and Uber
tesla_business_overview = tesla_sections.get("Item 1. Business", "")
uber_business_overview = uber_sections.get("Item 1. Business", "")
common_business_words = compare_business_overview(tesla_business_overview, uber_business_overview)

print("Content analysis complete.")

# Output results for verification
print("\nAlphabet's Risk Factors (sample):")
print("\n".join(alphabet_risk_factors[:5]))  # Show the first 5 risk factors

print("\nTesla's Risk Factors (sample):")
print("\n".join(tesla_risk_factors[:5]))  # Show the first 5 risk factors

print("\nCommon words in Tesla and Uber's business overviews (sample):")
print(common_business_words.most_common(10))  # Show the top 10 common words

Content analysis complete.

Alphabet's Risk Factors (sample):


Tesla's Risk Factors (sample):
You should carefully consider the risks described below together with the other information set forth in this report, which could materially affect our
business, financial condition and future results.
The risks described below are not the only risks facing our company.
Risks and uncertainties not currently
known to us or that we currently deem to be immaterial also may materially adversely affect our business, financial condition and operating results.

Risks Related to Our Ability to Grow Our Business

Risks Related to Our Operations
We may experience issues with lithium-ion cells or other components manufactured at our Gigafactories, which may harm the
production and profitability of our vehicle and energy storage products.


Common words in Tesla and Uber's business overviews (sample):
[('and', 326), ('to', 185), ('our', 151), ('the', 151), ('of', 119), ('in', 110), ('a', 68), ('we', 67),

In [12]:
def chatbot():
    while True:
        query = input("Ask your question (type 'exit' to quit): ").lower()
        if query == "exit":
            break
        elif "risk factors" in query:
            print("\nGoogle's Risk Factors:\n", "\n".join(alphabet_risk_factors[:5]), "\n")  # Showing only the first 5 for brevity
            print("Tesla's Risk Factors:\n", "\n".join(tesla_risk_factors[:5]), "\n")
        elif "total revenue" in query:
            print("Total revenue data extraction not implemented yet.")
        elif "business overview" in query:
            print("Common words in Tesla and Uber's business overview:\n", common_business_words.most_common(10), "\n")
        else:
            print("I'm sorry, I don't understand the question. Try asking about risk factors or business overviews.")

# Example usage:
# chatbot()